In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

np.random.seed(20160612)
tf.set_random_seed(20160612)

/Users/admin/.pyenv/versions/anaconda3-2.4.0/envs/tensorFlow/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
mnist = input_data.read_data_sets("tmp/data/", one_hot=True)

Extracting tmp/data/train-images-idx3-ubyte.gz
Extracting tmp/data/train-labels-idx1-ubyte.gz
Extracting tmp/data/t10k-images-idx3-ubyte.gz
Extracting tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
class SingleLayerNetwork:
    def __init__(self, num_units):
        with tf.Graph().as_default():
            self.prepare_model(num_units)
            self.prepare_session()
            
    def prepare_model(self, num_units):
        with tf.name_scope('input'):
            x = tf.placeholder(tf.float32, [None, 784])
            
        with tf.name_scope('hidden'):
            w1 = tf.Variable(tf.truncated_normal([784, num_units]))
            b1 = tf.Variable(tf.zeros([num_units]))
            hidden1 = tf.nn.tanh(tf.matmul(x, w1) + b1)
        
        with tf.name_scope('output'):
            w0 = tf.Variable(tf.zeros([num_units, 10]))
            b0 = tf.Variable(tf.zeros([10]))
            p = tf.nn.softmax(tf.matmul(hidden1, w0) + b0)
            
        with tf.name_scope('optimizer'):
            t = tf.placeholder(tf.float32, [None, 10])
            loss = -tf.reduce_sum(t * tf.log(p))
            train_step = tf.train.AdamOptimizer().minimize(loss)
        
        with tf.name_scope('evaluator'):
            correct_prediction = tf.equal(tf.argmax(p, 1), tf.argmax(t, 1))
            accuracy = tf.reduce_mean (tf.cast(correct_prediction, tf.float32))
            
#         tf.scalar_summary("loss",loss)
#         tf.scalar_summary("accuracy",accuracy)
#         tf.histogram_summary("weights_hidden",w1)
#         tf.histogram_summary("biases_hidden",b1)
#         tf.histogram_summary("weights_hidden",w0)
#         tf.histogram_summary("biases_hidden",b0)
        
        tf.summary.scalar("loss",loss)
        tf.summary.scalar("accuracy",accuracy)
        tf.summary.histogram("weights_hidden",w1)
        tf.summary.histogram("biases_hidden",b1)
        tf.summary.histogram("weights_hidden",w0)
        tf.summary.histogram("biases_hidden",b0)
        
        self.x, self.t, self.p = x, t, p
        self.train_step = train_step
        self.loss=loss
        self.accuracy = accuracy
    
    def prepare_session(self):
        sess = tf.InteractiveSession()
        sess.run(tf.global_variables_initializer())
#         summary = tf.merge_all_summaries()
        summary = tf.summary.merge_all()
#         writer = tf.train.SummaryWrighter("/tmp/mnist_st_logs", sess.graph)
        writer = tf.summary.FileWriter("/tmp/mnist_sl_logs", sess.graph)        
        self.sess = sess
        self.summary = summary
        self.writer = writer
        

In [4]:
!rm -rf /tmp/mnist_sl_logs


In [5]:
nn  =SingleLayerNetwork(1024)

i = 0
for _ in range(2000):
    i += 1
    batch_xs, batch_ts = mnist.train.next_batch(100)
    nn.sess.run(nn.train_step, feed_dict={nn.x:batch_xs, nn.t:batch_ts})
    if i % 100 == 0:
        summary, loss_val, acc_val = nn.sess.run([nn.summary, nn.loss, nn.accuracy], feed_dict={nn.x:mnist.test.images, nn.t:mnist.test.labels})
        print('Step:%d, Loss: %f, Accuracy: %f' % (i, loss_val, acc_val) )
        nn.writer.add_summary(summary, i)


Step:100, Loss: 3295.413330, Accuracy: 0.904800
Step:200, Loss: 2708.133057, Accuracy: 0.924000
Step:300, Loss: 2368.020508, Accuracy: 0.933300
Step:400, Loss: 2163.022461, Accuracy: 0.937900
Step:500, Loss: 2034.045410, Accuracy: 0.940200
Step:600, Loss: 1861.392578, Accuracy: 0.945300
Step:700, Loss: 1795.880981, Accuracy: 0.949100
Step:800, Loss: 1726.360474, Accuracy: 0.950000
Step:900, Loss: 1632.606812, Accuracy: 0.950400
Step:1000, Loss: 1528.746826, Accuracy: 0.953900
Step:1100, Loss: 1504.484619, Accuracy: 0.956200
Step:1200, Loss: 1459.716064, Accuracy: 0.957000
Step:1300, Loss: 1406.968506, Accuracy: 0.957200
Step:1400, Loss: 1388.481445, Accuracy: 0.957800
Step:1500, Loss: 1343.758789, Accuracy: 0.958800
Step:1600, Loss: 1335.115601, Accuracy: 0.960500
Step:1700, Loss: 1296.522949, Accuracy: 0.961300
Step:1800, Loss: 1243.706299, Accuracy: 0.961900
Step:1900, Loss: 1309.800171, Accuracy: 0.961500
Step:2000, Loss: 1228.673584, Accuracy: 0.962200
